Script de medición y carga de las colecciones en cluster MongoDB
TFM Daniel Herranz Segundo

In [1]:
import pandas as pd
import numpy as np
import os
import json
import random
import pymongo
from pymongo import MongoClient
import time
from pprint import pprint
from pymongo import InsertOne, DeleteMany, ReplaceOne, UpdateOne
import matplotlib.pyplot as plt
import psutil
import uuid

In [2]:
#Variables de los ficheros de datos salida
CurrentAccountCollection_file_out = '../MockData/MongoDB/CurrentAccountCollection/CurrentAccountCollection.json'
PositionKeepingCollection_file_out = '../MockData/MongoDB/PositionKeepingCollection/PositionKeepingCollection.json'
CustomerProfileCollection_file_out = '../MockData/MongoDB/CustomerProfileCollection/CustomerProfileCollection.json'

CurrentAccountCollection_sample_out = '../MockData/MongoDB/CurrentAccountCollection/CurrentAccountCollection_sample.json'
PositionKeepingCollection_sample_out = '../MockData/MongoDB/PositionKeepingCollection/PositionKeepingCollection_sample.json'
CustomerProfileCollection_sample_out = '../MockData/MongoDB/CustomerProfileCollection/CustomerProfileCollection_sample.json'

In [3]:
def save_results_to_csv(results,file,write_concerns):
    #Guardamos los resultados en csv
    from datetime import datetime
    dia = datetime.now().strftime("%d%m%Y_%H_%M_%S") 
    data = results
    results_df = pd.DataFrame(data, columns =['Registros', 'Tiempo', 'CPU','Memoria'])
    results.to_csv(file.format(write_concerns, str(dia)))

In [4]:
resultados_etl_CutomerProfileCollection = '../Results/MongoDB/MongoDB_Bulk_Insert_{}_WC_CustomerProfileCollection_{}.csv'
resultados_etl_PositionKeepingCollection = '../Results/MongoDB/MongoDB_Bulk_Insert_{}_WC_PositionKeepingCollection_{}.csv'
resultados_etl_CurrentAccountCollection = '../Results/MongoDB/MongoDB_Bulk_Insert_{}_WC_CurrentAccountCollection_{}.csv'

In [5]:
#repeticiones
repeats = 1000
test_wc = False

In [6]:
#Conexion unica a colecciones
connection = MongoClient('localhost', 27017, w=3) #Conexion con WriteConcern a 3 (primario y dos nodos consolidados)

#Creamos una base de datos para el trabajo llamada TFM
tfm_mongo_db = connection["tfm_mongo_database"]

## Creación de las colecciones por cada dominio

In [7]:
CustomerProfileCollection = tfm_mongo_db["CustomerProfileCollection"]
CurrentAccountCollection = tfm_mongo_db["CurrentAccountCollection"]
PositionKeepingCollection = tfm_mongo_db["PositionKeepingCollection"]
print(connection.list_database_names())

['admin', 'config', 'local', 'tfm_mongo_database']


## Carga de los documentos por cada dominio

### Test de carga CustomerProfileCollection

In [8]:
#Carga de la información de dataframes por entidades
CustomerProfileCollection_df = pd.read_json(CustomerProfileCollection_sample_out) #Carga Sample
CurrentAccountCollection_df = pd.read_json(CurrentAccountCollection_sample_out) #Carga Sample
PositionKeepingCollection_df = pd.read_json(PositionKeepingCollection_sample_out) #Carga Sample

#CustomerProfileCollection_df = pd.read_json(CustomerProfileCollection_file_out) 
#CurrentAccountCollection_df = pd.read_json(CurrentAccountCollection_file_out) 
#PositionKeepingCollection_df = pd.read_json(PositionKeepingCollection_file_out) 

In [9]:
print("CustomerProfileCollection_df",len(CustomerProfileCollection_df))

CustomerProfileCollection_df 1000000


In [10]:
#for x in range(17): # Generamos un millon de registros aprox
    #CustomerProfileCollection_df = CustomerProfileCollection_df.append(CustomerProfileCollection_df.sample(frac=0.40,replace = True))
    #CurrentAccountCollection_df = CurrentAccountCollection_df.append(CurrentAccountCollection_df.sample(frac=0.40,replace = True))
    #PositionKeepingCollection_df = PositionKeepingCollection_df.append(PositionKeepingCollection_df.sample(frac=0.40,replace = True))

In [11]:
num_documentos = 1000000

In [12]:
CustomerProfileCollection_df = CustomerProfileCollection_df.sample(num_documentos)
print(len(CustomerProfileCollection_df))
CustomerProfileCollection_df.sample(1)

1000000


,PartyId,PartyNumber,PartyType,Name,FullLegalName,LegalStructure,BeneficialOwnership,AccountRole,EmailAddress,Phone,Address
312910,f55057de-bb34-11ec-bad7-8faeae2e42b2,7166,sol,Kaela,Elliott McCullough V,Dr.,9,DE.OBIE.Principal,murphy.clarabelle@example.net,006.239.9487,"{'AddressType': 'New', 'AddressLine': '102 Sta..."


In [13]:
sample_mode = False #En cargas masivas reales, poner a false

if(sample_mode):
    list_partyId = CustomerProfileCollection_df['PartyId'].map(lambda x: str(uuid.uuid1()))
    list_accountid = CurrentAccountCollection_df['AccountId'].map(lambda x: str(uuid.uuid1()))
    CustomerProfileCollection_df['PartyId'] = list_partyId
    CurrentAccountCollection_df['AccountId'] = list_accountid
    CurrentAccountCollection_df['PartyId'] = list_partyId    
    PositionKeepingCollection_df['AccountId'] = list_accountid

print("CustomerProfileCollection_df",len(CustomerProfileCollection_df))
print("CurrentAccountCollection_df",len(CurrentAccountCollection_df))
print("PositionKeepingCollection_df",len(PositionKeepingCollection_df))

CustomerProfileCollection_df 1000000
CurrentAccountCollection_df 1000000
PositionKeepingCollection_df 1000000


In [14]:
#Cargas Masiva con Many
def loadCollectionDataFrame(df,collection):
    registers = []
    grps = df.groupby(df.index / 1000)    
    iter = 0;
    for i,group_df in grps: 
        #Insertamos en bloques de 1000 para igualar a otros motores    
        time_inicial = time.time()
        collection.insert_many(group_df.to_dict('records'))
        time_final = time.time()
        data_time_collection = round(time_final - time_inicial,3)
        used_cpu = psutil.cpu_percent()
        mem_used = psutil.virtual_memory().percent
        registers.append((iter,data_time_collection,used_cpu,mem_used))
        iter += 1000;
    return registers

In [15]:
#Borramos los valores actuales de la colección
connection["tfm_mongo_database"].CustomerProfileCollection.drop()
print('Borrada colleción CustomerProfileCollection')

Borrada colleción CustomerProfileCollection


In [16]:
CustomerProfileCollection.count_documents({})

0

In [17]:
registers_customer_profile = loadCollectionDataFrame(CustomerProfileCollection_df,CustomerProfileCollection)

In [18]:
CustomerProfileCollection.count_documents({})

1000000

In [19]:
data = registers_customer_profile
results_CustomerProfile_df = pd.DataFrame(data, columns =['Registros', 'Tiempo', 'CPU','Memoria'])
write_concerns = 3
save_results_to_csv(results_CustomerProfile_df,resultados_etl_CutomerProfileCollection, write_concerns)
results_CustomerProfile_df.sample(5)

,Registros,Tiempo,CPU,Memoria
315943,315943000,0.002,0.0,71.2
817300,817300000,0.002,0.0,71.2
517763,517763000,0.002,50.0,71.2
477359,477359000,0.002,0.0,71.2
919738,919738000,0.002,0.0,66.2


# CurrentAccountCollection ETL Load

In [20]:
CurrentAccountCollection_df = CurrentAccountCollection_df.sample(num_documentos)
print(len(CurrentAccountCollection_df))
CurrentAccountCollection_df.sample(1)

1000000


,AccountId,PartyId,Status,StatusUpdateDateTime,AccountType,NickName,OpeningDate,AccountSubType,AccountInfo
599276,a125b220-bb35-11ec-bad7-8faeae2e42b2,f5e7f558-bb34-11ec-bad7-8faeae2e42b2,Enabled,2011-09-10 19:07:30.00,Business,productize customized channels,1986-06-01 16:46:25.00,POI,"[{'SchemeName': 'UK.business', 'Identification..."


In [21]:
#Borramos los valores actuales de la colección
connection["tfm_mongo_database"].CurrentAccountCollection.drop()
print('Borrada colleción CurrentAccountCollection')

Borrada colleción CurrentAccountCollection


In [22]:
CurrentAccountCollection.count_documents({})

0

In [23]:
registers_CurrentAccount = loadCollectionDataFrame(CurrentAccountCollection_df,CurrentAccountCollection)

In [24]:
CurrentAccountCollection.count_documents({})

1000000

In [25]:
data = registers_CurrentAccount
results_CurrentAccount_df = pd.DataFrame(data, columns =['Registros', 'Tiempo', 'CPU','Memoria'])
write_concerns = 3
save_results_to_csv(results_CurrentAccount_df,resultados_etl_CurrentAccountCollection, write_concerns)
results_CurrentAccount_df.sample(5)

,Registros,Tiempo,CPU,Memoria
765797,765797000,0.002,0.0,67.2
693118,693118000,0.002,0.0,66.8
323698,323698000,0.002,0.0,64.4
454466,454466000,0.002,0.0,66.0
738916,738916000,0.002,100.0,67.2


## PositionKeepingCollection ETL Load

In [26]:
PositionKeepingCollection_df = PositionKeepingCollection_df.sample(num_documentos)
print(len(PositionKeepingCollection_df))
PositionKeepingCollection_df.sample(1)

1000000


,DateTime,CreditDebitIndicator,Type,Amount,CreditLine,AccountId
138099,1997-03-17 05:41:46,Credit,Business,"{'Currency': {'Code': 'EUR', 'Description': 'U...","{'Included': 0, 'Type': None, 'Amount': {'Curr...",a030a7b2-bb35-11ec-bad7-8faeae2e42b2


In [27]:
#Borramos los valores actuales de la colección
connection["tfm_mongo_database"].PositionKeepingCollection.drop()
print('Borrada colleción PositionKeepingCollection')

Borrada colleción PositionKeepingCollection


In [28]:
PositionKeepingCollection.count_documents({})

0

In [29]:
registers_PositionKeeping = loadCollectionDataFrame(PositionKeepingCollection_df,PositionKeepingCollection)

In [30]:
PositionKeepingCollection.count_documents({})

1000000

In [31]:
data = registers_PositionKeeping
results_pk_df = pd.DataFrame(data, columns =['Registros', 'Tiempo', 'CPU','Memoria'])
write_concerns = 3
save_results_to_csv(results_pk_df,resultados_etl_PositionKeepingCollection,write_concerns)
results_pk_df.sample(5)

,Registros,Tiempo,CPU,Memoria
478289,478289000,0.001,0.0,67.7
483323,483323000,0.002,0.0,67.7
808633,808633000,0.001,0.0,68.1
679278,679278000,0.001,0.0,68.0
471214,471214000,0.002,0.0,67.7


## Metricas con Write Concern = 2 (2 nodos de replica inmediata)

In [32]:
if(test_wc):
    write_concerns = 2

    #Conexion unica a colecciones
    connection = MongoClient('localhost', 27017, w=write_concerns) #Conexion con WriteConcern a 3 (primario y dos nodos consolidados)

    #Creamos una base de datos para el trabajo llamada TFM
    tfm_mongo_db = connection["tfm_mongo_database"]

In [33]:
if(test_wc):
    #Borramos los valores actuales de la colección
    connection["tfm_mongo_database"].CustomerProfileCollection.drop()
    print('Borrada colleción CustomerProfileCollection')
    connection["tfm_mongo_database"].CurrentAccountCollection.drop()
    print('Borrada colleción CurrentAccountCollection')
    connection["tfm_mongo_database"].PositionKeepingCollection.drop()
    print('Borrada colleción PositionKeepingCollection')

In [34]:
if(test_wc):
    registers = loadCollectionDataFrame(CustomerProfileCollection_df,CustomerProfileCollection)
    data = registers
    results_df = pd.DataFrame(data, columns =['Registros', 'Tiempo', 'CPU','Memoria'])
    save_results_to_csv(results_df,resultados_etl_CutomerProfileCollection, write_concerns)

In [35]:
if(test_wc):
    registers = loadCollectionDataFrame(CurrentAccountCollection_df,CurrentAccountCollection)
    data = registers
    results_df = pd.DataFrame(data, columns =['Registros', 'Tiempo', 'CPU','Memoria'])
    save_results_to_csv(results_df,resultados_etl_CurrentAccountCollection, write_concerns)

In [36]:
if(test_wc):
    registers = loadCollectionDataFrame(PositionKeepingCollection_df,PositionKeepingCollection)
    data = registers
    results_df = pd.DataFrame(data, columns =['Registros', 'Tiempo', 'CPU','Memoria'])
    save_results_to_csv(results_df,resultados_etl_PositionKeepingCollection, write_concerns)

##  Metricas con Write Concern = 1 (sin replica inmediata)


In [37]:
if(test_wc):
    write_concerns = 1

    #Conexion unica a colecciones
    connection = MongoClient('localhost', 27017, w=write_concerns) #Conexion con WriteConcern a 3 (primario y dos nodos consolidados)

    #Creamos una base de datos para el trabajo llamada TFM
    tfm_mongo_db = connection["tfm_mongo_database"]

In [38]:
if(test_wc):
    #Borramos los valores actuales de la colección
    connection["tfm_mongo_database"].CustomerProfileCollection.drop()
    print('Borrada colleción CustomerProfileCollection')
    connection["tfm_mongo_database"].CurrentAccountCollection.drop()
    print('Borrada colleción CurrentAccountCollection')
    connection["tfm_mongo_database"].PositionKeepingCollection.drop()
    print('Borrada colleción PositionKeepingCollection')

In [39]:
if(test_wc):
    registers = loadCollectionDataFrame(CustomerProfileCollection_df,CustomerProfileCollection)
    data = registers
    results_df = pd.DataFrame(data, columns =['Registros', 'Tiempo', 'CPU','Memoria'])
    save_results_to_csv(results_df,resultados_etl_CutomerProfileCollection, write_concerns)

In [40]:
if(test_wc):
    registers = loadCollectionDataFrame(CurrentAccountCollection_df,CurrentAccountCollection)
    data = registers
    results_df = pd.DataFrame(data, columns =['Registros', 'Tiempo', 'CPU','Memoria'])
    save_results_to_csv(results_df,resultados_etl_CurrentAccountCollection, write_concerns)

In [41]:
if(test_wc):
    registers = loadCollectionDataFrame(PositionKeepingCollection_df,PositionKeepingCollection)
    data = registers
    results_df = pd.DataFrame(data, columns =['Registros', 'Tiempo', 'CPU','Memoria'])
    save_results_to_csv(results_df,resultados_etl_PositionKeepingCollection, write_concerns)

In [42]:
#Cierre de la conexion
connection.close()